In [1]:
import time
import pycurl
import urllib
import json
import requests
import numpy as np

In [3]:
API_ENDPOINT_URL = 'http://lauzhack.sqpub.ch/'
API_KEY = '98423jhHJDmM'

class SwissQuoteStream:
    def __init__(self):
        
        self.conn = None
        self.buffer = ''
        
        
    def setup_connection(self):
        if self.conn:
            self.conn.close()
            self.buffer = ''
        self.conn = pycurl.Curl()
        self.conn.setopt(pycurl.URL, API_ENDPOINT_URL + 'prices')
        self.conn.setopt(pycurl.WRITEFUNCTION, self.callback)
    
    def start(self, callback):
        self.callback = callback
        self.setup_connection()
        backoff_network_error = 0.25
        backoff_http_error = 5
        backoff_rate_limit = 60
        while True:
            self.setup_connection()
            try:
                self.conn.perform()
            except:
                print('Network error')
                time.sleep(backoff_network_error)
                backoff_network_error = min(backoff_network_error + 1, 16)
                continue
            # HTTP Error
            sc = self.conn.getinfo(pycurl.HTTP_CODE)

            # HTTP error, use exponential back off up to 320 seconds
           # print(HTTP error %s, %s' % (sc, self.conn.errstr()))
           # print 'Waiting %s seconds' % backoff_http_error
            time.sleep(backoff_http_error)
            backoff_http_error = min(backoff_http_error * 2, 320)
            
    def buy(self, btc_amount):
        #data = str.encode('BUY 0.00000001 BTC ' + API_KEY)
        data = str.encode('BUY ' + str(btc_amount) +' BTC ' + API_KEY)
        r = requests.post(API_ENDPOINT_URL, data=data)
        return str(r.status_code) + ' ' + str(r.reason)

        
    def sell(self, btc_amount):
        #data = str.encode('SELL 0.00000001 BTC ' + API_KEY)
        data = str.encode('SELL ' + str(btc_amount) +' BTC ' + API_KEY)
        r = requests.post(API_ENDPOINT_URL, data=data)
        return str(r.status_code) + ' ' + str(r.reason)
        


In [4]:
stream = SwissQuoteStream()

In [5]:
def extract_price(data):
    return float(data.decode("utf-8").split(' ')[1][:-1])

WINDOW_MAX_SIZE = 20

window = []
epsilon = 0.01
ignore_count = 0
ignore_n = 1

normalizer_max = 5

def diff_normalizer(diff):
    if(diff > normalizer_max):
        diff_tmp = normalizer_max
    elif(diff < -normalizer_max):
        diff_tmp = -normalizer_max
    else:
        diff_tmp = diff
        
    normalized = (diff_tmp + normalizer_max) / (normalizer_max * 2)
    
    return normalized

def handle_new_price(data):
    global ignore_count

    new_price = extract_price(data)
    window.append(new_price)
    window_resized = window[-WINDOW_MAX_SIZE:]
    window_size = len(window_resized)
    
    weights = np.exp(np.linspace(-1., 0., window_size))
    weights /= weights.sum()
    
    a = np.convolve(window_resized, weights, mode='full')[:len(window_resized)]
    ema = a[window_size -1]
    
    diff = new_price - ema
    # diff > 0 -> price is above ema -> SELL SELL SELL!

    ignore_count = (ignore_count + 1 ) % ignore_n
    
    if(ignore_count == 0):
        big_enough = abs(diff) > epsilon
        
        if(not big_enough):
            print('Just HODL. Diff is: ' + str(diff))
            return
        
        END = 'http://lauzhack.sqpub.ch/teams/'

        r = requests.get(END)
        assets = r.json()[12]["assets"]
        cash = r.json()[12]["cash"]
        total = assets + cash
        
        current_percent_btc = assets / total #percentage of current bitcoin compared to total balance
        normalized_diff = 1 - diff_normalizer(diff) #percentage of bitcoin we want to have, compared to total balance
        percent_btc_diff = normalized_diff - current_percent_btc #percentage of cash to add
        amount_of_btc = percent_btc_diff * total # amount of bitcoin we want to add
        amount_in_btc = amount_of_btc / new_price #amount of equivalent bitcoin we want to add in cash
        
        #print(str(total) + ' ' + str(assets) + ' ' + str(cash) + ' ||| ' + str(diff) + ' ' + str(current_percent_btc) + ' ' + str(percent_btc_diff) + ' ' + str(amount_of_btc) + ' ' + str(amount_in_btc))
        
        if(amount_in_btc < 0):
            aa = stream.sell(abs(amount_in_btc))
            print('Diff: ' + str(normalized_diff) + '. SELL ' + str(abs(amount_in_btc)))
        else:
            aa = stream.buy(abs(amount_in_btc))
            print('Diff: ' + str(normalized_diff) + '. BUY ' + str(abs(amount_in_btc)))


In [ ]:
stream.start(handle_new_price)

Just HODL. Diff is: 0.0
Diff: 0.7975408415024049. BUY 5.617512552333753
Diff: 0.6345340068035512. SELL 0.8117720746584347
Diff: 0.5985227880370985. SELL 2.2627842486848198
Diff: 0.4048897184896759. SELL 0.060845420661297486
Diff: 0.4103955555089669. SELL 0.540712088693741
Diff: 0.41763707019913454. SELL 0.24000510942443795
Diff: 0.42366151367191374. BUY 4.641069495367862
Diff: 0.4291614227665377. SELL 0.6746303504561924
Diff: 0.43408245432988224. BUY 7.553162912027371
Diff: 0.43750976000437736. SELL 0.2473988362092819
Diff: 0.4414455659449231. BUY 7.34293711550932
Diff: 0.4449522168074964. BUY 0.08460351918815544
Diff: 0.4193266942230821. BUY 4.5854851117267135
Diff: 0.4223404206731175. BUY 0.07541605048667839
Diff: 0.4259990916317292. BUY 0.09816923104662541
Diff: 0.42934393401533266. BUY 0.08971340702517623
Diff: 0.4062797655419217. SELL 0.6194908885940886
Diff: 0.21020640211090724. SELL 5.261941928035591
Diff: 0.1166399268034638. SELL 2.508267072252182
Diff: 0.11343127484951765. SEL

ValueError: could not convert string to float: '3780.0\n2018-11-25T12:34:58.477000'

Network error
Diff: 0.24133149699487144. SELL 1.3212824460193453
Diff: 0.029794077020005694. SELL 4.023582717378651
Diff: 0.054524604024118184. BUY 0.30173121102543754
Diff: 0.0. SELL 1.4632891238279093
Diff: 0.0055377955524363864. SELL 1.3146550679529656
Diff: 0.0. SELL 0.1482470610362682
Diff: 0.0. SELL 3.91670945093514e-07
Diff: 0.012719985300464032. SELL 5.91430401872968
Diff: 0.03752398676178925. SELL 7.441035705788293
Diff: 0.04307813719983644. SELL 0.6597848727288275
Diff: 0.0. SELL 1.1552024469218716
Diff: 0.03320251369941618. BUY 0.8905628384682949
Diff: 0.08310474156915004. BUY 1.3367440875653254
Diff: 0.1202629641983094. BUY 0.9940418985764403
Diff: 0.08538222141878582. SELL 0.937324142854019
Diff: 0.0. SELL 2.284042744997913
Diff: 0.0. SELL 2.284024681756041
Diff: 0.0. SELL 6.301628520669715
Diff: 0.0. SELL 4.0107217474933154
Diff: 0.0. SELL 4.010700592558681
Diff: 0.0. SELL 2.685772477118956
Diff: 0.0. SELL 1.4795462218148872
Diff: 0.0. SELL 1.478754538872425
Diff: 0.0. SE

KeyboardInterrupt: 

Network error
